# 1. **Setting Up**

In [ ]:
pip install pickle5


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from random import sample
import re
from collections import OrderedDict
import numpy as np
from collections import Counter
import time
import pickle5 as pickle


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
main_link = "/content/drive/MyDrive/Research/Masters Thesis/V1/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2. **Create Class**

## 2.1 Required Functions

In [ ]:
# number = sample(range(0,2400),1)
# c = full_list[number[0]]
# print(number)
# test = c.split("connections\n\n\n\nSearch on Bing")[1] #1153
# test1 = test.split("Education\n\n\n\n")



# edu_splitter2 = ["\n\n\n\nSee fewer education\n\n","\nVolunteer Experience\n\n\n\n" , "\n\nSkills (" , "\n\n\n\nInterests\n\n\n\n" , "\n\n\n\nAccomplishments"  ]


# for i1 in full_list:
#     a = i1.split("connections\n\n\n\nSearch on Bing")
#     if len(a) != 2: a = i1.split("connections\n\n\n\nProfile")
#     b = a[1].split("\nEducation\n\n\n\n\tLogo")


#     counter_edu_splitter = 0
#     while counter_edu_splitter < len(edu_splitter2):
#       edu_splitter2_i = edu_splitter2[counter_edu_splitter]
#       counter_edu_splitter = counter_edu_splitter + 1 
#       if edu_splitter2_i in b[1]:
#         c = b[1].split(edu_splitter2_i)
#         c[1] = edu_splitter2_i + c[1]
#         break

In [ ]:
from enum import unique
def break_header(text):
  output = []
  t_split = text.split("\n\n")

  i = 0 
  while i < len(t_split):
      item = t_split[i]
      l_item = item.lower()
  #Sex
      if "sex =" in l_item or "sex=" in l_item : 
        if "m" in l_item: output.append("M") 
        else: output.append("F")
  #Name, uses the sex if statement 
        output.append(t_split[t_split.index(item)+ 1].strip())
  #Degree Connection 
      if "degree connection" in l_item: 
        output.append(l_item[-3:] )
        break 
      i += 1 

  #Current Location 
  output.append(t_split[-3].replace("·", "").strip()) 
  #Current Industry
  output.append(t_split[-2].replace("·", "")[:-4].strip())
  #Number of Connections 
  output.append(t_split[-1].strip())



  return output


def break_edu(text):

  associate_degree_list = ["Associates","Associate","Associate’s","Associate's", "AA", "A.A.", "AS","A.S.","AAS","A.S","A.A.A."]
  bachelor_degree_list = ["Bachelors","Bachelor","Bachelor’s", "Bachelor's" , "B.A.","B.A", "BA", "BM", "B.M.", "BS","B.S","B.S.","BBA","B.B.A.","B.Eng","B.Eng.","BTech","B.Tech.","B.Tech", "BSE","Btech","BSEE","Barchelor","Bsc"]
  grad_degree_list =  ["Postgraduate","Master","Masters", "Master’s", "Master's", "M.A.","MS", "MA", "Juris Doctor","J.D.","JD" ,"M.B.A.","MBA", "PhD", "Ph.D.", "Doctor", "Ed.D.", "EdD"]
  total_degree_list = associate_degree_list + bachelor_degree_list + grad_degree_list
  set_degree_list = set(total_degree_list)

  text = text.replace("•", "")
  split_edu = text.split("\n\n\tSchool name\n\n")

  edu_details = []
  for i in split_edu[1:]:
    edu_details_i = []
    split_content = set(re.split('\s', i))
    is_degree_check = set(set_degree_list).intersection(split_content)
    if len(is_degree_check) > 0: 
          # Extract : Name of School, Degree, Dates, Field
          split_content2 = i.split("\n\n")
          split_content2 = [x.strip() for x in split_content2]

          #Name of school 
          edu_details_i.append(split_content2[0])

          #Degree 
          for j in [grad_degree_list, bachelor_degree_list , associate_degree_list]: 
            if len(set(j).intersection(is_degree_check))>0: #To only get the highest one if multiple degree attained 
              edu_details_i.append(j[0])
              break 


          #Dates
          if 'Dates attended or expected graduation' in split_content2:  
            date_index = split_content2.index('Dates attended or expected graduation')
            if date_index < len(split_content2)-1:edu_details_i.append(split_content2[date_index + 1])  #Some pofile do not have dates for degrees
            else: edu_details_i.append(None)
          else: 
            edu_details_i.append(None)

          #Field 
          if 'Field of study' in split_content2: 
            field_index = split_content2.index('Field of study')
            edu_details_i.append(split_content2[field_index + 1])
          else:
            edu_details_i.append(None)

                
          #Degree Name 
          if 'Degree name' in split_content2: 
            degreeName_index = split_content2.index('Degree name')
            edu_details_i.append(split_content2[degreeName_index + 1])
          else: 
            edu_details_i.append(None)


          edu_details.append(edu_details_i)  

  #if len(edu_details) == 0 : print(text)
  return edu_details


def break_interests(text):
  text_break = text.split("\n\n\n\n")
  clean_text_break = []


  for i in text_break:

    stripped_i = i.strip()
    if stripped_i != "" : 
      a = []
      b = stripped_i.split("\n\n")
      for j in b: 
          if j.strip()!="": a.append(j.strip())

      clean_text_break.append(a)
    
  #Remove repeated values and remove string "follower"
  interest_w_follower_raw = [x for x in clean_text_break if len(x)>1]
  interest_wo_follower = [x for x in clean_text_break if x not in interest_w_follower_raw ]
  interest_w_follower = [[x[0], int(re.sub('\D', '', x[-1]).strip())] for x in interest_w_follower_raw] #Remove string follower
  interest_w_follower_names = [[x[0]] for x in interest_w_follower]
  non_repeated_interest = [[x[0],None] for x in interest_wo_follower if x not in interest_w_follower_names]
  
  a = interest_w_follower + non_repeated_interest
  res = list(k for k,_ in itertools.groupby(a))

  output = res
  return output 



def break_text(text):

  #Header
  header_requests = ["sex", "name", "DegConn", "Cloc", "CInd", "CComp", "CPos", "NCon"]
  header_splitter1 = "connections\n\n\n\nSearch on Bing"
  header_splitter2 = "connections\n\n\n\nProfile"
  header_split = text.split(header_splitter1)
  if len(header_split) != 2: header_split = text.split(header_splitter2) #At least one case
  header_item_list = break_header(header_split[0])

  #Education
        # Top Split 
  edu_split_top = header_split[1].split("\nEducation\n\n\n\n\tLogo")  
        # Bottom Split
  edu_splitter2 = ["\n\n\n\nSee fewer education\n\n","\nVolunteer Experience\n\n\n\n" , "\n\nSkills (" , "\n\n\n\nInterests\n\n\n\n" , "\n\n\n\nAccomplishments"  ]
  counter_edu_splitter = 0                                         
  while counter_edu_splitter < len(edu_splitter2):               
      edu_splitter2_i = edu_splitter2[counter_edu_splitter]
      if edu_splitter2_i in edu_split_top[1]:
        edu_split_top_bottom = edu_split_top[1].split(edu_splitter2_i)
        edu_split_top_bottom[1] = edu_splitter2_i + edu_split_top_bottom[1]
        break
      counter_edu_splitter = counter_edu_splitter + 1 
  
      #Get stuff 
  
  edu_item_list = break_edu(edu_split_top_bottom[0])


  #Language 
  #Skip for now

  #Skills 
  skill_split_top = edu_split_top_bottom[1].split("\n\nSkills (")   #Top Split 
  if len(skill_split_top) == 1:  #Skill list does not exist in profile
    skill_output = [0,[],[]]
  else:  
    skill_split_bottom = skill_split_top[1].split("\n\n\n\nShow fewer\n\n")  #Bottom split 
    number_skills, skill_list, skill_endors_pair = break_skills(skill_split_bottom[0])
    skill_output = [number_skills, skill_list, skill_endors_pair]
  
  #Interests 
  if len(skill_split_top) == 1 : Interests_split = skill_split_top  #592 no skills list so len(skill_split_top) == 1
  else: Interests_split = skill_split_top[1].split("\nInterests\n\n\n\n")

  if len(Interests_split) == 1: Intrests_output = [[None,None]]
  else: Intrests_output = break_interests(Interests_split[1])

  #Summary 
  summary_split_top = edu_split_top[0].split("\n\nSummary\n\n\n")
  if len(summary_split_top) == 1 : summary_output = None 
  elif len(summary_split_top) ==2 : 
    summary_split_bottom = summary_split_top[1].split("\n\nExperience\n\n\n")
    summary_output = summary_split_bottom[0].replace("See less of summary", "").strip()


  #Employment 
  employment_split_top = edu_split_top[0].split("\n\nExperience\n\n\n")
  employment_split_bottom = employment_split_top[1].replace("See fewer positions", "")
  #print([employment_split_bottom])
  employment_output = break_employment(employment_split_bottom)


  output = header_item_list
  output.append(edu_item_list) 
  output.append(skill_output)
  output.append(Intrests_output)
  output.append(summary_output)
  output.append(employment_output)

  return output

  
def get_att(diced_text, request):
  if request == "sex": return diced_text[0]
  elif request == "name": return diced_text[1]
  elif request == "DegConn": return diced_text[2]
  elif request == "Cloc": return diced_text[3]
  elif request == "CInd": return diced_text[4]
  elif request == "NCon": return diced_text[5]
  elif request[:-1] == "edu":
    edu_index = int(request[-1:])
    if edu_index <= len(diced_text[6]): return diced_text[6][edu_index-1]
    else: return [None, None, None, None, None]
  elif request == "NSkills": return diced_text[7][0]
  elif request == "SkillsList": return diced_text[7][1]
  elif request == "SkillEndor": return diced_text[7][2]
  elif request == "Interests": return diced_text[8]
  elif request == "Summary": return diced_text[9]
  elif request == "Employment" : return diced_text[10]     ############  Update this later       #############
  else: return None


def break_skills(text):

  text_break = [x.strip() for x in text.split("\n") if x !="" ]

  #Removing company names from skills list
  visited = set()
  dup_pre = list(set([x for x in text_break if x in visited or (visited.add(x) or False)]))
  duplicates = [x for x in dup_pre  if x[-12:] != "endorsements" and x[-11:] != "endorsement"]
  cleaned_skills_split = [x for x in text_break if x not in duplicates] 

  #Attaching endorments
  len_clean_split = len(cleaned_skills_split)
  i = 1                     #First element is the number of skills 
  skill_list = []
  endorsement_list = []
  while i < len_clean_split:
    i_skill = cleaned_skills_split[i]
    if "endorsement" not in i_skill :    #Not a endorsement item 
      if i!= len_clean_split-1 and "endorsement" in cleaned_skills_split[i+1]: #next item is endorsement
        skill_list.append(i_skill)
        endorsement_list.append(int(cleaned_skills_split[i+1][:-12].strip()))
      else: 
        skill_list.append(i_skill)
        endorsement_list.append(0)
    i=i+1
        

  skill_endors_pair = zip(skill_list,endorsement_list )
  number_skills = re.sub('\D', '', cleaned_skills_split[0])

  return number_skills, skill_list, list(skill_endors_pair) 



def break_employment(text):

  # Find Grouped company names 
  split_every_line = text.split("\n\n")
  i = 0 
  len_split_every_line = len(split_every_line)
  group_company_name = []
  while i < len_split_every_line:
    if i != 0 : im1_split = split_every_line[i-1].strip()
    else: im1_split = ""
    i_split = split_every_line[i].strip()
    if i != len_split_every_line -1 : ip1_split = split_every_line[i+1].strip()
    else: ip1_split = ""

    if im1_split == i_split and ("yrs" in ip1_split  or "mos" in ip1_split ): group_company_name.append(im1_split)
    i = i+1 
  #print(group_company_name)
  
  #Split using group company name and Position title
  position_split = text.split("\n\tPosition title\n")
  main_emp_split = []
  for i in position_split: 
    if i == position_split[0]: current_i =  i 
    else: current_i = "\n\tPosition title\n" + i 

    len_pre_comp_name_check = len(main_emp_split)
    last_line_current_i = current_i.strip().split("\n\n")[-1]
    for j in group_company_name: #Company name split 
        split_w_comp_name = current_i.strip().split(j)
        if j in current_i and ("yrs" in last_line_current_i  or "mos" in last_line_current_i) and len(split_w_comp_name[-1].strip())<16 and len(split_w_comp_name[-1].strip()) > 4: 
          main_emp_split.append(split_w_comp_name[0])
          main_emp_split.append([j,split_w_comp_name[-1].strip()]) #Adding [Company name, years worked]
          break 
    len_post_comp_name_check = len(main_emp_split)
    
    if len_pre_comp_name_check == len_post_comp_name_check : main_emp_split.append(current_i)


  all_jobs = []
  current_g_company = []
  current_g_company_duration = []
  company_list = []


  for job in main_emp_split:
    job_details_list = [ None, [None, None], [None, None], None, None, None] #Title, C name, Dates, Location, Position Summary, Skills
    if len(job) == 2:
      #Update Grouped company name 
      current_g_company = job[0]
      current_g_company_duration = job[1]
    elif len(job) > 10: 
      pre_list = job.strip().split("\n\n")
      pre_list = [x.strip() for x in pre_list]

      if 'Company name' not in pre_list : job_details_list[1] =  [current_g_company, None] #If company name not present use the group name
      summary_signal = 0
      i = 0 
      while summary_signal == 0 and i < len(pre_list)-1:
        text_i = pre_list[i].strip()
        if text_i == 'Position title' : job_details_list[0] =  pre_list[i+1].strip()
        if text_i == 'Company name' : 
          job_details_list[1] =  pre_list[i+1].strip().split(" · ")
          if len(job_details_list[1]) == 1 : job_details_list[1].append(None) #If no employment status (full/part) available
        if text_i == 'Dates employed and Duration' : job_details_list[2] =  pre_list[i+1].split(" • ")
        if text_i == 'Position location': job_details_list[3] =  pre_list[i+1].strip()
        if text_i == 'Position summary' : job_details_list[4] =  pre_list[i+1:]

        #Now accounting for scenarioes where skills might occur 
        if text_i[0:7] == "Skills:" : 
          job_details_list[5] = text_i[7:].strip().split(" • ")
        if (i == len(pre_list)-2 and pre_list[i+1].strip()[0:7] == "Skills:" ): 
           job_details_list[5] =  pre_list[i+1].strip()[7:].strip().split(" • ")
      
        i = i+1    

      company_list.append(job_details_list[1][0])
      if job_details_list[0]!= None:  all_jobs.append(job_details_list)


  #Get rid of next company name at the end of summary 
 
  all_fixed_job = []
  for job in all_jobs:
    fixed_job = job 
    job_summary = fixed_job[4]
    if job_summary != None:  
      if len(job_summary) > 0 and job_summary[-1] in company_list: fixed_job[4] = fixed_job[4][:-1]
      single_description = ["\n" if item == '' else item for item in fixed_job[4]]
      fixed_description = ""
      for line in single_description : fixed_description = fixed_description + "\n" + line
      fixed_job[4] = fixed_description
    else: 
      fixed_description = None

    all_fixed_job.append(fixed_job)

  output = all_fixed_job
  return output 



#####################################
# Under Construction 
#####################################






#####################################
# Under Construction 
#####################################



## 2.2 Class Definition

In [ ]:

class Person:
    def __init__(self, text):
       diced_text = break_text(text)
       self.sex = get_att(diced_text, "sex")
       self.name = get_att(diced_text, "name")
       self.degree_connection = get_att(diced_text, "DegConn")
       self.current_location = get_att(diced_text, "Cloc")
       self.current_industry = get_att(diced_text, "CInd")
       self.number_connection = get_att(diced_text, "NCon")
       self.edu1 = self.Education(get_att(diced_text, "edu1"))
       self.edu2 = self.Education(get_att(diced_text, "edu2"))
       self.edu3 = self.Education(get_att(diced_text, "edu3"))
       self.edu4 = self.Education(get_att(diced_text, "edu4"))
       self.edu5 = self.Education(get_att(diced_text, "edu5"))
       self.edu6 = self.Education(get_att(diced_text, "edu6"))
       self.number_skills = get_att(diced_text, "NSkills")
       self.skills_list = get_att(diced_text, "SkillsList")
       self.skills_endorsement_pair = get_att(diced_text, "SkillEndor")
       self.Intersts = get_att(diced_text, "Interests")
       self.summary = get_att(diced_text, "Summary")
       self.employment = get_att(diced_text , "Employment")
                    #Here employment is a list of jobs where each job contains information below 
    
    class Education:
     def __init__(self,edu_info):
        self.SName = edu_info[0]
        self.SLevel = edu_info[1]
        self.SDates = edu_info[2]
        self.SField = edu_info[3]
        self.SDegree = edu_info[4]

    class Employment:
      def __init__(self, job_info):

        self.title = job_info[0]
        self.company_name = job_info[1]
        self.job_dates = job_info[2]
        self.job_location = job_info[3]
        self.job_summary = job_info[4]
        self.job_skills = job_info[5]

        
        
# sex, name, degree_connection, current_location, current_industry, number_connection, 
# edu1, edu2, edu3, edu4, edu5, edu6, 
# number_skills, skills_list, skills_endorsement_pair, Intersts, summary,
# employment

# Education is iteself and object with the following attributes
# SName, SLevel, SDates, SField, SDegree

#Here employment is a list of jobs where each job contains information below 
# [title, company_name, job_dates, job_location, job_summary, job_skills]

  

# 3. **Loading Files**

In [ ]:

with open(main_link + 'Pickle/Philadelphia.pkl', 'rb') as inp:
    object_list1 = pickle.load(inp)

with open(main_link  + 'Pickle/Austin.pkl', 'rb') as inp:
    object_list2 = pickle.load(inp)

with open(main_link  + 'Pickle/Atlanta_DC_LasVegas_Minneapolis_SanAntonio_SanDiego_VirginiaBeach.pkl', 'rb') as inp:
    object_list3 = pickle.load(inp)

with open(main_link  + 'Pickle/Fresno_Milwaukee_Tulsa.pkl', 'rb') as inp:
    object_list4 = pickle.load(inp)
    
object_list = object_list1 + object_list2 + object_list3 +object_list4

In [ ]:
print(len(object_list))

2401
